In [5]:
# this code ran in a python 3.7 virtual environment.
import os
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import cv2
# import matplotlib.pyplot as plt
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
!pip install opencv-python

     ---------------------------------------- 38.2/38.2 MB 9.6 MB/s eta 0:00:00


In [6]:
# Loading the labels
noisy = pd.read_csv('noisy_labels.csv',header=None)
clean = pd.read_csv('clean_labels.csv',header=None)

In [9]:
# loading the noisy images
noisy_images = []
for i in range(10001,50001):
    noisy_images.append(cv2.imread('images/{:05d}.png'.format(i)))
    if i % 1000 == 0:
        print(i, sep=" ")
noisy_images = torch.Tensor(np.stack(noisy_images))

11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000


In [40]:
# dataloader for noisy images
sz = 50
images_trial = torch.utils.data.DataLoader(noisy_images, batch_size=sz)
noisylabel_trial = torch.utils.data.DataLoader(nn.functional.one_hot(torch.Tensor(noisy[0].iloc[10000:].values).long()), batch_size=sz)

In [39]:
# loading the training data
images = []
for i in range(1,10001):
    images.append(cv2.imread('images/{:05d}.png'.format(i)))
    if i % 1000 == 0:
        print(i, sep=" ")
images = torch.Tensor(np.stack(images))

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [41]:
# dataloader for training, validation, and test data
sz = 50
images_trainloader = torch.utils.data.DataLoader(images[:6000], batch_size=sz)
noisylabel_trainloader = torch.utils.data.DataLoader(nn.functional.one_hot(torch.Tensor(noisy[0].iloc[:6000].values).long()), batch_size=sz)
label_trainloader = torch.utils.data.DataLoader(nn.functional.one_hot(torch.Tensor(clean[0].iloc[:6000].values).long()), batch_size=sz)

images_valoader = torch.utils.data.DataLoader(images[6000:8000], batch_size=sz)
noisylabel_valoader = torch.utils.data.DataLoader(nn.functional.one_hot(torch.Tensor(noisy[0].iloc[6000:8000].values).long()), batch_size=sz)
label_valoader = torch.utils.data.DataLoader(nn.functional.one_hot(torch.Tensor(clean[0].iloc[6000:8000].values).long()), batch_size=sz)

images_testloader = torch.utils.data.DataLoader(images[8000:10000], batch_size=sz)
noisylabel_testloader = torch.utils.data.DataLoader(nn.functional.one_hot(torch.Tensor(noisy[0].iloc[8000:10000].values).long()), batch_size=sz)
label_testloader = torch.utils.data.DataLoader(nn.functional.one_hot(torch.Tensor(clean[0].iloc[8000:10000].values).long()), batch_size=sz)



In [31]:
# for evaluating the model on test data
def evaluator_cuda(model,images_testloader, noisylabel_testloader, label_testloader):
  correct = 0
  total = 0
  for t, (t_d, t_nl, t_l) in enumerate(zip(images_testloader, noisylabel_testloader, label_testloader), 1):
    test_outputs = model(t_d.to(DEVICE), t_nl.to(DEVICE))
    correct += sum(torch.argmax(test_outputs, dim=1) == torch.argmax(t_l.to(DEVICE), dim=1))
    total += len(t_l)
  accuracy = correct / total
  print('Accuracy on the test set: %f' % (correct / total))
  return accuracy

In [42]:
nclass = 10

# defining the CNN to be used
class MyCNN7(nn.Module):
    def __init__(self,num_class):
        super(MyCNN7, self).__init__()
        # Define first convolution layer
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1) # 3 input channels, 64 output channels, and a 3x3 kernel with padding of 1
        self.bn1 = nn.BatchNorm2d(64) # Batch normalization for the output of the first convolution layer
        self.relu1 = nn.ReLU() # Activation function to introduce non-linearity
        self.pool1 = nn.MaxPool2d(2, 2) # Max pooling layer to reduce the spatial dimensions of the output
        
        # Define second convolution layer
        self.conv2 = nn.Conv2d(64, 64, 3, padding=1) # 64 input channels, 64 output channels, and a 3x3 kernel with padding of 1
        self.bn2 = nn.BatchNorm2d(64) # Batch normalization for the output of the second convolution layer
        self.relu2 = nn.ReLU() # Activation function to introduce non-linearity
        self.pool2 = nn.MaxPool2d(2, 2) # Max pooling layer to reduce the spatial dimensions of the output
        
        # Define fully connected layer
        self.fc1 = nn.Linear(64 * 8 * 8 + num_class, 256)
        self.bn3 = nn.BatchNorm1d(256) # Batch normalization for the output of the fully connected layer
        self.relu3 = nn.ReLU() # Activation function to introduce non-linearity
        
        # Define additional fully connected layer
        self.fcx = nn.Linear(256,128)
        self.bnx = nn.BatchNorm1d(128) # Batch normalization for the output of the additional fully connected layer
        self.relux = nn.ReLU() # Activation function to introduce non-linearity

        # Define final fully connected layer with softmax activation
        self.fc2 = nn.Linear(128, num_class)
        self.sm1 = nn.Softmax(1) # Apply softmax activation to the output to obtain a probability distribution over classes
        self.num_class = num_class

    def forward(self, x,y):
        # reformatting the inputs to fit the network
        y = torch.Tensor(y)
        x = x.permute(0,3,1,2)
        # implementing the layers
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.reshape(x.shape[0],64*8*8)
        x = torch.cat((x, y), dim=1)
        x = x.reshape(-1, 64 * 8 * 8 + self.num_class)
        x = self.fc1(x.float())
        x = self.bn3(x)        
        x = self.fcx(x)
        x = self.bnx(x)
        x = self.relux(x)
        x = self.relu3(x)
        x = self.fc2(x)
        x = self.sm1(x)
        return x

model = MyCNN7(nclass)
model = model.to(DEVICE)

# using adam as optimizer
# optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
# optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.7)
optimizer = optim.Adam(model.parameters(),lr = 0.00025)



# using cross entropy loss
criterion = nn.CrossEntropyLoss()
def custom_loss(pred, true, missclassified):
    loss_1 = criterion(pred, true)
    return loss_1
num_epochs = 75

# will tolerate three epochs without validation loss improvement
# training will stop early after a fourth
patience = 3
best_val_loss = float('inf')
epochs_since_improvement = 0

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (d, nl, l) in enumerate(zip(images_trainloader, noisylabel_trainloader, label_trainloader), 1):
        d = d.to(DEVICE)
        nl = nl.to(DEVICE)
        l = l.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(d, nl)
        # add .float() when using crossentropyloss
        #loss = criterion(outputs, l.float())
        loss = custom_loss(outputs, l.float(), nl.float())
        # backprop
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Epoch [{}/{}], Training Loss: {:.4f}'.format(epoch+1, num_epochs, running_loss/len(images_trainloader)))
    
    with torch.no_grad():
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        for j, (val_d, val_nl, val_l) in enumerate(zip(images_valoader, noisylabel_valoader, label_valoader), 1):
            val_d = val_d.to(DEVICE)
            val_nl = val_nl.to(DEVICE)
            val_l = val_l.to(DEVICE)
            val_outputs = model(val_d, val_nl)
            # calculate validation loss to monitor model performance
            val_loss += custom_loss(val_outputs, val_l.float(), val_nl.float())
            # val_loss += criterion(val_outputs, val_l.float()).item()
            _, predicted = torch.max(val_outputs.data, 1)
            _, gt = torch.max(val_l.data, 1)
            total += val_l.size(0)
            correct += (predicted == gt).sum().item()
        val_loss /= len(images_valoader)
        val_acc = 100 * correct / total
        print('Epoch [{}/{}], Validation Loss: {:.4f}, Validation Accuracy: {:.2f}%'.format(epoch+1, num_epochs, val_loss, val_acc))
    # saving the model with the lowest validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pt')
        epochs_since_improvement = 0
    else:
        epochs_since_improvement += 1
    # early stopping if validation loss is not improving
    if epochs_since_improvement > patience:
        print('Stopping training after {} epochs without improvement'.format(epochs_since_improvement))
        break

# evaluation on test data
testacc_2 = evaluator_cuda(model,images_testloader, noisylabel_testloader, label_testloader)

Epoch [1/75], Training Loss: 2.1314
Epoch [1/75], Validation Loss: 2.0636, Validation Accuracy: 46.90%
Epoch [2/75], Training Loss: 1.9924
Epoch [2/75], Validation Loss: 1.9748, Validation Accuracy: 49.10%
Epoch [3/75], Training Loss: 1.9185
Epoch [3/75], Validation Loss: 1.9519, Validation Accuracy: 51.15%
Epoch [4/75], Training Loss: 1.8745
Epoch [4/75], Validation Loss: 1.9211, Validation Accuracy: 54.45%
Epoch [5/75], Training Loss: 1.8300
Epoch [5/75], Validation Loss: 1.9109, Validation Accuracy: 54.90%
Epoch [6/75], Training Loss: 1.8018
Epoch [6/75], Validation Loss: 1.9095, Validation Accuracy: 55.55%
Epoch [7/75], Training Loss: 1.7652
Epoch [7/75], Validation Loss: 1.8762, Validation Accuracy: 58.75%
Epoch [8/75], Training Loss: 1.7514
Epoch [8/75], Validation Loss: 1.9032, Validation Accuracy: 55.90%
Epoch [9/75], Training Loss: 1.7298
Epoch [9/75], Validation Loss: 1.8958, Validation Accuracy: 56.55%
Epoch [10/75], Training Loss: 1.7064
Epoch [10/75], Validation Loss: 1.88

In [44]:
# load back the version of the model with minimal validation loss
checkpoint = torch.load("best_model.pt")
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [45]:
# evaluate on test data
evaluator_cuda(model,images_testloader, noisylabel_testloader, label_testloader)
# result: 60.45% accurately classified images

Accuracy on the test set: 0.604500


tensor(0.6045)

In [63]:
# applying the model on the other 40000 noisy images
preds = []
for t, (t_d, t_nl) in enumerate(zip(images_trial, noisylabel_trial), 1):
    preds.append(torch.argmax(model(t_d.to(DEVICE), t_nl.to(DEVICE)), dim=1))
    if t%80 == 0:
        print(t)
preds = torch.cat(preds)

80
160
240
320
400
480
560
640
720
800


In [70]:
# an updated set of labels for training on image classification
pd.concat([clean,pd.DataFrame(preds)]).to_csv("edited_labels.csv",index=False)